# Mule Deer Site-Scale Scoring Calculator 

Steps for the site-scale scoring calculator in python
1. Read in an Excel spreadsheet (or a csv)
2. Convert measures to scores
3. Aggregate scores by season functional class (e.g., forage)
4. Aggregate scores by season
5. Calculate functional acres per season
6. Append to Excel spreadsheet and save

## 1. Read in Excel Spreadsheet

Use the Pandas library to read Excel files into a dataframe for editing. Pandas is imported as pd by convention. We will also import numpy as np to provide access to functions in numpy that will be useful later. We always import all libraries at the beginning of the script.

In [1]:
import pandas as pd
import numpy as np

To import a spreadsheet, use the read_excel function. To learn more about the function, including which parameters are required, use a question mark before the function to access the help documentation. Run the cell below (select the cell and press Shift+Enter).

In [2]:
#?pd.read_excel

Now use the function to read in the data. Specify the full path to the workbook with the workbook_path variable in the cell below. Then read in the data. (The r before the path name indicates to python to ignore the backslashes, which would otherwise be read as an escape character. This is referred to as a raw string. See more at https://docs.python.org/2.0/ref/strings.html)

In [3]:
workbook_path = r'Book1.xlsx'
data = pd.read_excel (workbook_path)

To view the data, simply type the name of the variable and run the cell.

In [4]:
data

,Map_Unit_ID,Acres,Sage_CC
0,1,10,0.25
1,2,100,0.05
2,3,1000,0.10


To access a column, use bracket notation, as below.

In [5]:
sage_pct = data['Sage_CC']
sage_pct

0    0.25
1    0.05
2    0.10
Name: Sage_CC, dtype: float64

The data is returned as a pandas series. Use the built-in type function to check the type of any variable.

In [6]:
type(sage_pct)

pandas.core.series.Series

You can run calculations on each row and save the results in a new column very simply. Specify the new column to the left of the equal sign, and perform the calculation to the right of the equal sign.
For example, we might convert the percentages to whole values by multiplying each by 100, saving the results in a new column named 'Sage_Pct'. Don't forget the parentheses around column names.

In [7]:
data['Sage_Pct'] = data['Sage_CC'] * 100
data

,Map_Unit_ID,Acres,Sage_CC,Sage_Pct
0,1,10,0.25,25.0
1,2,100,0.05,5.0
2,3,1000,0.10,10.0


Suppose instead we wanted to apply our own function to the dataframe. We first must write our function.

In [8]:
def percent_converter(x):
    """Converts x from a percent to a whole number"""
    y = x * 100
    return y

Let's try out our function

In [9]:
percent_converter(.25)

25.0

Now, let's apply it to the Sage_CC column of the dataframe and store the results in a new column named 'Sage_Pct_Fun'. We'll need to use pandas map function to do this.

In [10]:
data['Sage_Pct_Fun'] = data['Sage_CC'].map(percent_converter)
data

,Map_Unit_ID,Acres,Sage_CC,Sage_Pct,Sage_Pct_Fun
0,1,10,0.25,25.0,25.0
1,2,100,0.05,5.0,5.0
2,3,1000,0.10,10.0,10.0


We might also want to average all of the values in the dataframe and return a series with those values. To do this, we'll use the apply function. Specify axis = 0 to perform the operation on each column, as opposed to each row (axis=1).

In [11]:
averages = data.apply(np.mean, axis=0)
averages

Map_Unit_ID       2.000000
Acres           370.000000
Sage_CC           0.133333
Sage_Pct         13.333333
Sage_Pct_Fun     13.333333
dtype: float64

To access a single row, we can use the notation below to specify which row based on the value for a specific column. In this case, we'll get the row for which Map_Unit_ID is equal to 2. 

In [12]:
data[data['Map_Unit_ID']==2]

,Map_Unit_ID,Acres,Sage_CC,Sage_Pct,Sage_Pct_Fun
1,2,100,0.05,5.0,5.0


Alternatively, we can use the .loc and .iloc methods of a dataframe to select single records and get a pandas series in return. .loc provides access based on the index and .iloc provides access based on the position of the index. To make this easier, we might want to change the index of our data to the Map_Unit_ID column. Hint: Use Shift+Enter after typing a method to access help documentation for the paramaters to use, here we use the inplace parameter to avoid creating a new copy of the data).

In [13]:
data.set_index('Map_Unit_ID', inplace=True)
data

,Acres,Sage_CC,Sage_Pct,Sage_Pct_Fun
Map_Unit_ID,,,,
1,10,0.25,25.0,25.0
2,100,0.05,5.0,5.0
3,1000,0.10,10.0,10.0


In [14]:
data.loc[2]

Acres           100.00
Sage_CC           0.05
Sage_Pct          5.00
Sage_Pct_Fun      5.00
Name: 2, dtype: float64

To delete a column, use the .drop() method.

In [15]:
data.drop('Sage_Pct_Fun', axis=1, inplace=True)
data

,Acres,Sage_CC,Sage_Pct
Map_Unit_ID,,,
1,10,0.25,25.0
2,100,0.05,5.0
3,1000,0.10,10.0


This should be enough to get you going on the site-scale scoring calculator. Let me know if you get stuck, but I know you can do it!

# 2. Convert Measures to Scores

In [16]:
##to create site scale veg score, need to include forb cover and desirable forb cover! 
data['Forb_Pct'] = [10,5,20] #forb cover
#desirable forb cover. we might need to create a function that subtracts invasives from forb cover  
data['D_Forb_Pct'] = [8,5,15] 
#desirable shrub cover (for winter scores)
data ['d_shrub_PCT'] = [20, 5, 5]

In [17]:
def summershrub(perc): ##from mule deer methods doc- score of summer shrub cover
    if perc <= 15:
        score = 0.2
    elif perc <= 30: 
        score = 0.5
    elif perc <= 50: 
        score = 0.8
    elif perc <= 60:
        score = 1.0
    else: 
        score = 0.5
    return score


In [18]:
#test 
summershrub(80)

0.5

In [19]:
##define function to convert forb cover to score
def summerforb(forb_CC): 
    if forb_CC <= 30: 
        score = 0.05
    elif forb_CC <= 60: 
        score = 0.33
    else: 
        score = 1
    return score

In [20]:
#test
summerforb(35)

0.33

In [21]:
##not incorporating 'desirable forbs' because the scoring curves are not decided yet

In [22]:
#define fucntion to convert desirable shrub cover to score
def desirable_shrub(dshrub):
    if dshrub <= 25: 
        score = .25
    elif dshrub <= 50: 
        score= .50
    elif dshrub <= 75:
        score = .75
    else: 
        score = 1
    return (score)


In [23]:
data

,Acres,Sage_CC,Sage_Pct,Forb_Pct,D_Forb_Pct,d_shrub_PCT
Map_Unit_ID,,,,,,
1,10,0.25,25.0,10,8,20
2,100,0.05,5.0,5,5,5
3,1000,0.10,10.0,20,15,5


In [24]:
##add score to new column 
data['Shrubcoverscore'] = data['Sage_Pct'].map(summershrub)
data['forbcoverscore'] = data['Forb_Pct'].map(summerforb)
data['dshrubscore'] = data['d_shrub_PCT'].map(desirable_shrub)
data

,Acres,Sage_CC,Sage_Pct,Forb_Pct,D_Forb_Pct,d_shrub_PCT,Shrubcoverscore,forbcoverscore,dshrubscore
Map_Unit_ID,,,,,,,,,
1,10,0.25,25.0,10,8,20,0.5,0.05,0.25
2,100,0.05,5.0,5,5,5,0.2,0.05,0.25
3,1000,0.10,10.0,20,15,5,0.2,0.05,0.25


# 3. Aggregated Based on Season

In [25]:
data['Sage_Pct'] = data['Sage_CC'] * 100

data['s_vegscore']= data['Shrubcoverscore'] * 0.50 + data['forbcoverscore'] * 0.25 + data['forbcoverscore'] * 0.25
#change the 2nd forb cover score to "desirable forb score" when we figure that out

data['w_vegscore']= data['Shrubcoverscore'] * 0.50 + data['dshrubscore'] * .50

# 4. Calculate functional acres per season

In [26]:
data['s_func_ac'] = data['Acres'] * data['s_vegscore']
data['w_func_ac'] = data['Acres'] * data['w_vegscore']

data

,Acres,Sage_CC,Sage_Pct,Forb_Pct,D_Forb_Pct,d_shrub_PCT,Shrubcoverscore,forbcoverscore,dshrubscore,s_vegscore,w_vegscore,s_func_ac,w_func_ac
Map_Unit_ID,,,,,,,,,,,,,
1,10,0.25,25.0,10,8,20,0.5,0.05,0.25,0.275,0.375,2.75,3.75
2,100,0.05,5.0,5,5,5,0.2,0.05,0.25,0.125,0.225,12.50,22.50
3,1000,0.10,10.0,20,15,5,0.2,0.05,0.25,0.125,0.225,125.00,225.00


final output

In [27]:
print(data[['s_func_ac', 'w_func_ac']])

print("Total Summer Functional Acres")
print(data['s_func_ac'].sum())
print("Total Winter Functional Acres")
print(data['w_func_ac'].sum())

             s_func_ac  w_func_ac
Map_Unit_ID                      
1                 2.75       3.75
2                12.50      22.50
3               125.00     225.00
Total Summer Functional Acres
140.25
Total Winter Functional Acres
251.25


# 5. Save

In [28]:
data.to_excel("func_acres_output.xlsx")

## other things/ drawing board

In [29]:
#summer/migration veg score
#weights
#desirable forbs .25
#forbs cover  .25
#shrub cover .5

#def summerveg(shrubcoverscore, forbscoverscore, dshrubscore): 
 #   summervegscore = shrubcoverscore * 0.50 + forbscoverscore * 0.25 + dshrubscore * 0.25
  #  return (summervegscore)

In [30]:
#summerveg(1,2,3)

In [31]:
#data['summervegscore'] = data['shrubcoverscore, forbscoverscore, dshrubscore'].map(summerveg)
#data